# Reading data from external databases

To import external data into a pandas dataframe, we use beautifulsoup4. The source is the JORDANUS database of medieval scientific documents, maintained by http://jordanus.badw.de.

In [1]:
import json
import sys
import urllib
from bs4 import BeautifulSoup
import re
import requests

import pandas as pd

url = 'http://jordanus.badw.de/cgi-bin/lrz_jord-search.pl?sprache=en&datenbank=ptolemaeus&ausgabe=dhstext&ausdateiformat=txt&listpos=0&listen=keine&listlet=keiner&fn=t52a56.f&fi=Martianus+Capella'
    
r  = requests.get(url)

data = r.text

soup = BeautifulSoup(data)

/usr/lib/python3/dist-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Since the data is organized in a html table, we have to search for the table environments and the included columns and rows. Each column is translated in a key:value pair of a dictionary. 

In [3]:
data = []
tables = soup.find_all('table')

for table in tables:
    item = {}
    table_body = table.find_all('tr')
    for row in table_body:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        if cols:
            item[cols[0]] = cols[1]
    data.append(item) 

The dictionary is read as a dataframe. 

In [4]:
df = pd.DataFrame(data)
df.keys()

Index(['10/3', '11/2', '12/3', '12/4', '13/2', '13/4', '13/5', '13/6', '14/3',
       '15/9',
       ...
       'Source', 'Source's source', 'Stype of writing', 'Subject 1',
       'Subject 2', 'T & K', 'Type of ms. 2', 'Type of source', 'Watermarks',
       'Width of columns'],
      dtype='object', length=182)

To reduce the needed columns, the keys are given as a list to the dataframe. 

In [5]:
df[['Author standard','Short title','Short title st.','Library','Language','Folio no.','City','Country','Century']].dropna(how='all').reset_index(drop=True)

,Author standard,Short title,Short title st.,Library,Language,Folio no.,City,Country,Century
0,Macrobius\nMartianus Capella,NaN,Selections from Macrobius (part.),The Library of Robert B. Honeyman Jr.,Latin,79r-80v,San Juan Capistrano,USA,12\n13\n14\n15
1,Martianus Capella\nMacrobius (attrib.),NaN,De nuptiis Philologiae et Mercurii (part.),Library of the Hispanic Society of America,Latin,1r-6v,New York,USA,NaN
2,Martianus Capella,NaN,Annotations de genre divers,Stedelijke Openbare Bibliotheek,Latin,309r-311r,Brugge,Belgium,14
3,Martianus Capella,Recueil de traités scientifiques,De nuptiis Philologiae et Mercurii,Bibliothèque Royale,Latin,13v-196v,Bruxelles,Belgium,10
4,Martianus Capella,NaN,"De nuptiis Philologiae et Mercurii (part., inc.)",Staatsbibliothek,Latin,65r-79r,Bamberg,Germany,10
5,Martianus Capella,NaN,De nuptiis Philologiae et Mercurii,Staatsbibliothek,Latin,NaN,Bamberg,Germany,10
6,Martianus Capella\nMacrobius (?),"Opera gromatica, astronomica, historica cum Ex...",Com. in somn. Scip. (?)\nArgumentum quo magnit...,Staatsbibliothek,Latin,28v-29,Bamberg,Germany,11\n9\n10
7,Martianus Capella,Mathematisch-astronomische Sammelhs.,"De nuptiis Mercurii cum Philologia, lib. VIII",Staatsbibliothek zu Berlin,Latin,3,Berlin,Germany,12/II\n12
8,Martianus Capella,NaN,De nuptiis Mercurii et Philologiae libri I et II,Bayerische Staatsbibliothek,Latin,46-55,München,Germany,11/12
9,NaN,NaN,Ars geometrica et astronomica,Bayerische Staatsbibliothek,Latin,48v-69v,München,Germany,9\n10
